There are N data points, $\mathbf{z_1} ... \mathbf{z_N}$. 
 
$$p(\mathbf{z_n}) = N (\mathbf{z_n}|\mathbf{0},I)$$

$$p(\mathbf{z_n}|\mathbf{x_n}) = N (\mathbf{z_n}|M^{-1} W^T \mathbf{x_n}, \sigma^2 M)$$

In the algorithm we calculate an estimate of $E[\mathbf{z_n}]$. I.e: $M^{-1} W^T x_n$. Using the values of W (and M) generated in the M-step. We find $E[\mathbf{z_n}]$ for each of the z's. For some reason the distribution of these expectations is normally distributed with mean 0, variance 1...

Below we run the EM algorithm to estimate W etc, I've recorded the mean and std of the E[z] latent variables.

In [2]:
#First set up our dataset, etc: A 3d dataset
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

Dn = 3 #number of dimensions in the observed variables.
Mn = 2 #number of dimensions the latent variables will have (principal components)
N = 200 #number of data points

#initial values of parameters sigma^2 and the transform W (between the observed space and the latent space)
sigsqr = 1.0 #starting value of the noise variance sigma^2
W = np.random.randn(Dn,Mn) * .01 #starting value of the transform W

#data generation
x = np.random.randn(Dn,N)
x[1,:] += x[0,:]*1.5 #some covariance
x[0,:] = x[0,:] * 5.0
x[1,:] = x[1,:] * 2.0
x[2,:] -= x[1,:] * 4
x = x + np.random.randn(Dn,N) * .0

x = x * 10.0
#used later for efficiency.
zeromean_x = (x.T - np.mean(x,1)).T

Ezz = []
Ez = []
for n in range(N):
    Ezz.append(np.zeros([Mn,Mn]))
    Ez.append(np.zeros(Mn)[:,None])

expz_mean = []
expz_std = []
expzz_
for it in range(10000):
    #Expectation, find values of components...
    M = np.dot(W.T,W) + sigsqr * np.eye(Mn)
    invM = np.linalg.inv(M) #TODO Solve using linalg.solve (MORE STABLE)
    old_Ez = Ez[:] #Use Cholesky decomp to do this inversion in a more stable way.

    for n in range(N): #TODO: Vectorise
        Ez[n] = np.dot(np.dot(invM,W.T),zeromean_x[:,n])[:,None] #MxD . Dx1 = Mx1
        Ezz[n] = sigsqr * invM + np.dot(Ez[n],Ez[n].T)
    #Maximisation 
    #Next We've differentiated E[ln p(X,Z|parameters)] and,
    #setting equal to zero, we can find what the parameters equal to maximise this    
    
    partA = np.zeros([Dn,Mn])
    for n in range(N):
        partA += np.dot(zeromean_x[:,n][:,None],Ez[n].T)
        #if n==5:
        #    print np.dot(zeromean_x[:,n][:,None],Ez[n].T)
        #    print np.dot(zeromean_x[:,n][:,None],np.dot(np.dot(invM,W.T),zeromean_x[:,n])[:,None].T)
        #    print np.dot(np.dot(zeromean_x[:,n][:,None],zeromean_x[:,n][:,None].T),np.dot(W,invM.T))
        #    print "----"

        
    partB = np.zeros([Mn,Mn])
    for n in range(N):
        partB += Ezz[n]
    new_W = np.dot(partA, np.linalg.inv(partB)) #TODO Replace inverse
    temp = 0
    for n in range(N):
        #np.linalg.norm() replace below
        temp += np.dot(zeromean_x[:,n],zeromean_x[:,n]) - 2.0*np.dot(np.dot(Ez[n].T, new_W.T),zeromean_x[:,n])
        temp += np.trace(np.dot(Ezz[n],np.dot(new_W.T,new_W)))
    new_sigsqr = (1.0/(N*Dn)) * temp #changed brackets

    sig_store.append(new_sigsqr)
   
    W = new_W.copy()
    sigsqr = new_sigsqr
    expz = np.array(Ez[:]).squeeze()
    expz_std.append(np.std(expz,0))
    expz_mean.append(np.mean(expz))
print W
import matplotlib.pyplot as plt
%matplotlib inline

NameError: name 'sig_store' is not defined

Below I've plotted how the standard deviation (and mean) of the $E[z_n]$ vectors converge towards N(0,1).

In [ ]:
plt.plot(np.vstack(expz_std))
plt.plot([0,10000],[1,1],'--k')
plt.title('E[z] standard deviation across points, over iterations')
plt.ylim([.9,1.4])

plt.figure()
plt.plot(expz_mean)
plt.plot([0,10000],[0,0],'--k')
plt.title('E[z] standard deviation across points, over iterations')
#plt.ylim([min(expz_mean),max(expz_mean)])

Neil explained that *"p(z|x) will tend towards N(0,1) after convergence, as this is expected for whitened data"*. 

Intuitively this makes sense: Our data fits the prior most closely when this is true, and due to the number of degrees of freedom available (through the scale of W's values, for example) the z values can move towards this distribution.

array([[ 18.245383,  -9.768028],
       [ -9.768028,   6.532717]])